In [ ]:
from automaticTB.examples import get_Si_structure_2NN
import os 
from automaticTB.tools import read_yaml, write_yaml
from automaticTB.interaction import (
    get_InteractingAOSubspaces_from_cluster, CombinedAOSubspaceInteraction
)

bulksi = get_Si_structure_2NN()
orbitals = {"Si":"2s2p3s"}
all_interaction_pairs = []
for i, cluster in enumerate(bulksi.nnclusters):
    temporary_file = os.path.join("tmp_yaml", f"si_2nn_tmp_{i+1}.yaml")
    if os.path.exists(temporary_file):
        iaosubspace = read_yaml(temporary_file)
    else:
        iaosubspace = get_InteractingAOSubspaces_from_cluster(cluster, orbitals)
        write_yaml(iaosubspace, temporary_file)
    all_interaction_pairs += iaosubspace

combined = CombinedAOSubspaceInteraction(all_interaction_pairs)

free_aos = combined.free_AOpairs
for i, free in enumerate(free_aos):
    print(f"{i+1:>2d} " + str(free))